## PDS API demo
# Visualization of Osiris-Rex OVIRS spectrometer observations of asteroid BENNU

The purpose of this notebook is to demostrate how the PDS web API can be used to access the PDS data for a scientific use case.

The documention of the PDS web API is available on https://nasa-pds.github.io/pds-api/

This notebook is available on https://github.com/NASA-PDS/pds-api-notebook

 
<b>WARNING:</b> This notebook is a demo and not a real scientific use case. It might contain mistakes in the way the data is used or displayed.

In [1]:
from __future__ import print_function
from pprint import pprint
import time
import matplotlib.pyplot as plt
from astropy.io import fits

The PDS API is accessed using a python client library documented on https://nasa-pds.github.io/pds-api-client/

In [2]:
import pds.api_client as pds_api

## Use the PDS demo web API server 

Connect to the demo server. See User Interface of the web API: https://pds-gamma.jpl.nasa.gov/api/swagger-ui.html

<b>Note: </b> this piece of code will be wrapped into a helper function so that 1 line will be enough to connect to the API using a default host

In [3]:
configuration = pds_api.Configuration()

# demo server
configuration.host = 'https://pds.nasa.gov/api/search/1/'

api_client = pds_api.ApiClient(configuration)


## List requestable properties

Available properties can be identified with request

In [11]:
from pds.api_client.apis.paths.products import Products
products = Products(api_client)
help(products)
api_response = products.get(query_params={"limit": 1},
                            accept_content_types=("application/json",))
#print(api_response.body["summary"]["properties"])
#products_api = pds_api.ProductsApi(api_client)
#
#api_response = products_api.products(only_summary=True)
#print(f"Available properties are {', '.join(api_response.summary.properties[1:10])}...")

Help on Products in module pds.api_client.apis.paths.products object:

class Products(pds.api_client.paths.products.get.ApiForget)
 |  Products(api_client: Union[pds.api_client.api_client.ApiClient, NoneType] = None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator
 |  Ref: https://openapi-generator.tech
 |  
 |  Do not edit the class manually.
 |  
 |  Method resolution order:
 |      Products
 |      pds.api_client.paths.products.get.ApiForget
 |      pds.api_client.paths.products.get.BaseApi
 |      pds.api_client.api_client.Api
 |      builtins.object
 |  
 |  Methods inherited from pds.api_client.paths.products.get.ApiForget:
 |  
 |  get(self, query_params: pds.api_client.paths.products.get.RequestQueryParams = frozendict.frozendict({}), accept_content_types: Tuple[str] = ('*', '*/*', 'application/csv', 'application/json', 'application/kvp+json', 'application/vnd.nasa.pds.pds4+json', 'application/vnd.nasa.pds.pds4+xml', 'application/xml', 'text/csv', 'text/html', 

## Get observations around specific spot (lat=12, lon=24) with distance to bennu closer than 4 km

Get the result found in part1 by directly posting the search criteria to the API

We can use `ProductsApi.products` with argument `q=`.

The query syntax is described in the PDS API specification. It uses the following operators:
- comparison: lt, le, ...
- boolean: and, or, not
- groups: (, )

See https://github.com/NASA-PDS/pds-api/blob/master/docs/spec/pds-api-specification.md#query-syntax for detailed syntax.

We request the FITS data file url from the API in the `field=` argument.

In [34]:
start_time = time.time() 

criteria = ' ( ( ref_lid_target eq "urn:nasa:pds:context:target:asteroid.101955_bennu" )'
criteria += ' and ( ref_lid_instrument eq "urn:nasa:pds:context:instrument:ovirs.orex" ) '
#criteria += ' and ( orex:spatial.orex:target_range lt 400.0 )'
#criteria += ' and ( orex:spatial.orex:target_range lt 4.0 )'
criteria += ' ) '
#criteria += " and ( orex:spatial.orex:latitude ge 9.0 ) and ( orex:spatial.orex:latitude le 15.0 )"
#criteria += " and ( orex:spatial.orex:longitude ge 21.0 ) and ( orex:spatial.orex:longitude le 27.0 ) )"

#criteria2 = '(ref_lid_target eq "urn:nasa:pds:context:target:asteroid.101955_bennu")'
properties_of_interest = ['orex:spatial.orex:latitude', 'orex:spatial.orex:longitude', 'ref_lid_instrument', 'orex:spatial.orex:target_range', 'ops:Data_File_Info.ops:file_ref']
start = 0
limit = 10

api_response = products.get(query_params={"q": criteria,
                                          "start": start, "limit": limit,
                                          "fields": properties_of_interest},
                            accept_content_types=("application/json",)).body

#closer_products = products_api.products(q=criteria, fields=properties_of_interest)

elapsed = time.time() - start_time
#print(f'retrieved {len(closer_products.data)} products in {elapsed} seconds')
print(api_response)

DynamicSchema({'summary': DynamicSchema({'q': ' ( ( ref_lid_target eq "urn:nasa:pds:context:target:asteroid.101955_bennu" ) and ( ref_lid_instrument eq "urn:nasa:pds:context:instrument:ovirs.orex" )  ) ', 'hits': Decimal('10000'), 'took': Decimal('567'), 'start': Decimal('0'), 'limit': Decimal('10'), 'sort': (), 'properties': ('lidvid', 'ops:Data_File_Info/ops:file_ref', 'ops:Label_File_Info/ops:file_ref', 'ops:Tracking_Meta/ops:archive_status', 'orex:spatial/orex:latitude', 'orex:spatial/orex:longitude', 'orex:spatial/orex:target_range', 'pds:File/pds:creation_date_time', 'pds:Modification_Detail/pds:modification_date', 'pds:Time_Coordinates/pds:start_date_time', 'pds:Time_Coordinates/pds:stop_date_time', 'product_class', 'ref_lid_instrument', 'ref_lid_instrument_host', 'ref_lid_investigation', 'ref_lid_target', 'title', 'vid')}), 'data': (DynamicSchema({'id': 'urn:nasa:pds:orex.mission::9.0', 'type': 'Product_Bundle', 'title': 'Origins, Spectral Interpretation, Resource Identificatio

## Structure of the reponse

The responses are JSON objects, as follow

In [ ]:
pprint(closer_products.data[0])

### Check that we have the right lidvids

In [ ]:
lidvids = [product.id for product in closer_products.data]
print(f'The lidvids of the selected products are {lidvids}')

### Extract the links to the data files

In [ ]:
data_files = [product.properties['ops:Data_File_Info.ops:file_ref'] for product in closer_products.data]
print(f"product data file url: {data_files}")

## Plot the data (FITS files)

Plot the spectra of the 4 observations on the same figure, one figure per dimension of the instrument.


In [ ]:
hduls = [fits.open(data_file) for data_file in data_files]

for i in range(20): # for each dimension of the instrument
    fig, ax = plt.subplots()
    ax.set_title(f'detector {i}')
    for hdul in hduls: # for each observation
        ax.plot(hdul[2].data[0, i, :], hdul[0].data[0, i, :])
    ax.set_xlabel('wavelength (micrometers)')
    ax.set_ylabel('radiance')